In [16]:
import os
import sys
import numpy as np
from numpy.lib.arraysetops import unique
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from copy import deepcopy

from datetime import datetime
from pprint import pprint
import tqdm
import ipdb
import pickle

plt.style.use("seaborn")
np.random.seed(1)

from training.utils import load_obj, save_obj
from training.data import load_data
from training.dataset import _preprocess_call_data, preprocess_and_make_dataset

from sklearn.cluster import KMeans, OPTICS, SpectralClustering
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from training.modelling.metrics import F1, Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import load_model
from training.modelling.dataloader import get_train_val_test


In [4]:

# stats = pd.read_csv("may_data/beneficiary_stats_v5.csv")
# beneficiary_data = pd.read_csv("may_data/beneficiary/AIRegistration-20200501-20200731.csv")
# b_data, call_data = load_data("may_data")
# call_data = _preprocess_call_data(call_data)
# all_beneficiaries = stats[stats['Group'].isin(["Google-AI-Control", "Google-AI-Calls"])]
# transitions = pd.read_csv("may_data/RMAB_one_month/weekly_transitions_SI_single_group.csv")

In [18]:
pilot_trans_df = pd.read_csv('outputs/pilot_transitions.csv')

In [19]:
# with open('may_data/features_dataset.pkl', 'rb') as fw:
#     features_dataset = pickle.load(fw)
# fw.close()

In [22]:
def get_transition_probabilities(beneficiaries, transitions, min_support=3):
    transitions = transitions[transitions['user_id'].isin(beneficiaries)]

    i_transitions = transitions[transitions['action']=='Intervention']
    n_i_transitions = transitions[transitions['action']=='No Intervention']

    i_L = i_transitions[i_transitions['pre-action state']=="L"]
    i_H = i_transitions[i_transitions['pre-action state']=="H"]

    i_L_L = i_L[i_L['post-action state']=="L"]
    i_L_H = i_L[i_L['post-action state']=="H"]

    i_H_L = i_H[i_H['post-action state']=="L"]
    i_H_H = i_H[i_H['post-action state']=="H"]

    n_i_L = n_i_transitions[n_i_transitions['pre-action state']=="L"]
    n_i_H = n_i_transitions[n_i_transitions['pre-action state']=="H"]

    n_i_L_L = n_i_L[n_i_L['post-action state']=="L"]
    n_i_L_H = n_i_L[n_i_L['post-action state']=="H"]

    n_i_H_L = n_i_H[n_i_H['post-action state']=="L"]
    n_i_H_H = n_i_H[n_i_H['post-action state']=="H"]

    transition_probabilities = dict()
    if i_L.shape[0] >= min_support:
        transition_probabilities['P(L, I, L)'] = i_L_L.shape[0] / i_L.shape[0]
        transition_probabilities['P(L, I, H)'] = i_L_H.shape[0] / i_L.shape[0]
    else:
        transition_probabilities['P(L, I, L)'] = np.nan
        transition_probabilities['P(L, I, H)'] = np.nan

    if i_H.shape[0] >= min_support:
        transition_probabilities['P(H, I, L)'] = i_H_L.shape[0] / i_H.shape[0]
        transition_probabilities['P(H, I, H)'] = i_H_H.shape[0] / i_H.shape[0]
    else:
        transition_probabilities['P(H, I, L)'] = np.nan
        transition_probabilities['P(H, I, H)'] = np.nan
    
    if n_i_L.shape[0] >= min_support:
        transition_probabilities['P(L, N, L)'] = n_i_L_L.shape[0] / n_i_L.shape[0]
        transition_probabilities['P(L, N, H)'] = n_i_L_H.shape[0] / n_i_L.shape[0]
    else:
        transition_probabilities['P(L, N, L)'] = np.nan
        transition_probabilities['P(L, N, H)'] = np.nan

    if n_i_H.shape[0] >= min_support:
        transition_probabilities['P(H, N, L)'] = n_i_H_L.shape[0] / n_i_H.shape[0]
        transition_probabilities['P(H, N, H)'] = n_i_H_H.shape[0] / n_i_H.shape[0]
    else:
        transition_probabilities['P(H, N, L)'] = np.nan
        transition_probabilities['P(H, N, H)'] = np.nan

    return transition_probabilities, {'P(L, I, L)': i_L_L.shape[0], 'P(L, I, H)': i_L_H.shape[0], 'P(H, I, L)': i_H_L.shape[0], 'P(H, I, H)': i_H_H.shape[0], 'P(L, N, L)': n_i_L_L.shape[0], 'P(L, N, H)': n_i_L_H.shape[0], 'P(H, N, L)': n_i_H_L.shape[0], 'P(H, N, H)': n_i_H_H.shape[0]}

def get_all_transition_probabilities(train_beneficiaries, transitions):
    cols = [
        "P(L, I, L)", "P(L, I, H)", "P(H, I, L)", "P(H, I, H)", "P(L, N, L)", "P(L, N, H)", "P(H, N, L)", "P(H, N, H)", 
    ]
    transition_probabilities = pd.DataFrame(columns = ['user_id'] + cols)
    user_ids = train_beneficiaries['user_id']

    for user_id in tqdm.tqdm(user_ids):
        probs, _ = get_transition_probabilities([user_id], transitions, min_support=1)
        probs['user_id'] = user_id

        transition_probabilities = transition_probabilities.append(probs, ignore_index=True)

    return transition_probabilities

def get_individual_transition_clusters(train_transitions, n_clusters):
    cols = [
        "P(L, I, L)", "P(L, I, H)", "P(H, I, L)", "P(H, I, H)", "P(L, N, L)", "P(L, N, H)", "P(H, N, L)", "P(H, N, H)", 
    ]
    
    train_beneficiaries = pd.DataFrame()
    train_beneficiaries['user_id'] = train_transitions['user_id'].unique()

    all_transition_probabilities = get_all_transition_probabilities(train_beneficiaries, train_transitions)
    pass_to_kmeans_cols = ['P(L, N, L)', 'P(H, N, L)']
    train_labels, centroids, _, cls, num_clusters, max_iters = kmeans_missing(all_transition_probabilities[pass_to_kmeans_cols], n_clusters, max_iter=100)

    train_beneficiaries['cluster'] = train_labels


    cluster_transition_probabilities = pd.DataFrame(columns=['cluster', 'count'] + cols)

    for i in range(n_clusters):
        cluster_beneficiaries = train_beneficiaries[train_beneficiaries['cluster'] == i]
        cluster_b_user_ids = cluster_beneficiaries['user_id']
        probs, _ = get_transition_probabilities(cluster_b_user_ids, train_transitions, min_support=3)
        print(i, probs, len(cluster_b_user_ids))
        probs['cluster'] = i
        probs['count'] = len(cluster_b_user_ids)
        cluster_transition_probabilities = cluster_transition_probabilities.append(probs, ignore_index=True)


    return cluster_transition_probabilities, all_transition_probabilities, train_labels

def kmeans_missing(X, n_clusters, max_iter=10):
    n_clusters = CONFIG['clusters']
    missing = ~np.isfinite(X)
    mu = np.nanmean(X, 0, keepdims=1)
    X_hat = np.where(missing, mu, X)

    prev_labels = None
    for i in range(max_iter):
        if CONFIG['clustering'] == 'optics':
            cls = OPTICS(min_samples=4, n_jobs=-1)
        elif CONFIG['clustering'] == 'kmeans':
            cls = KMeans(n_clusters, n_jobs=-1, random_state=0)
        elif CONFIG['clustering'] == 'spectral':
            cls = SpectralClustering(n_clusters, n_jobs=-1, random_state=0)

        labels = cls.fit_predict(X_hat)

        if CONFIG['clustering'] == 'kmeans':
            centroids = cls.cluster_centers_
        else:
            if CONFIG['clustering'] == 'optics':
                labels = labels + 1
            unique_labels = len(set(labels))
            centroids = []
            for i in range(unique_labels):
                idxes = np.where(labels == i)[0]
                centroids.append(np.mean(X_hat[idxes], axis=0))
            centroids = np.array(centroids)

        X_hat[missing] = centroids[labels][missing]

        if i > 0 and np.all(labels == prev_labels):
            break

        prev_labels = labels

    return labels, centroids, X_hat, cls, len(set(labels)), i

In [26]:
aug_states = []
for i in range(6):
    if i % 2 == 0:
        aug_states.append('L{}'.format(i // 2))
    else:
        aug_states.append('H{}'.format(i // 2))
CONFIG = {
    "problem": {
        "orig_states": ['L', 'H'],
        "states": aug_states + ['L', 'H'],
        "actions": ["N", "I"],
    },
#     "time_step": 7,
    "gamma": 0.99,
    "clusters": 40,
    "transitions": "weekly",
    "clustering": "kmeans",
}

cluster_transition_probabilities, all_transition_probabilities, train_labels = get_individual_transition_clusters(pilot_trans_df,  CONFIG['clusters']) 


100%|██████████| 23003/23003 [03:00<00:00, 127.61it/s]


0 {'P(L, I, L)': 0.6666666666666666, 'P(L, I, H)': 0.3333333333333333, 'P(H, I, L)': 0.42857142857142855, 'P(H, I, H)': 0.5714285714285714, 'P(L, N, L)': 0.65569774527727, 'P(L, N, H)': 0.34430225472273, 'P(H, N, L)': 0.21706263498920086, 'P(H, N, H)': 0.7829373650107991} 324
1 {'P(L, I, L)': 0.7164179104477612, 'P(L, I, H)': 0.2835820895522388, 'P(H, I, L)': nan, 'P(H, I, H)': nan, 'P(L, N, L)': 1.0, 'P(L, N, H)': 0.0, 'P(H, N, L)': 1.0, 'P(H, N, H)': 0.0} 195
2 {'P(L, I, L)': 0.5384615384615384, 'P(L, I, H)': 0.46153846153846156, 'P(H, I, L)': 0.1339031339031339, 'P(H, I, H)': 0.8660968660968661, 'P(L, N, L)': 0.0, 'P(L, N, H)': 1.0, 'P(H, N, L)': 0.10364161849710983, 'P(H, N, H)': 0.8963583815028902} 1793
3 {'P(L, I, L)': 0.9777043765483072, 'P(L, I, H)': 0.022295623451692816, 'P(H, I, L)': nan, 'P(H, I, H)': nan, 'P(L, N, L)': 1.0, 'P(L, N, H)': 0.0, 'P(H, N, L)': 0.2545454545454545, 'P(H, N, H)': 0.7454545454545455} 3649
4 {'P(L, I, L)': nan, 'P(L, I, H)': nan, 'P(H, I, L)': 0.021

In [28]:
all_transition_probabilities

,user_id,"P(L, I, L)","P(L, I, H)","P(H, I, L)","P(H, I, H)","P(L, N, L)","P(L, N, H)","P(H, N, L)","P(H, N, H)"
0,2394384.0,1.0,0.0,NaN,NaN,1.0,0.0,1.000000,0.000000
1,2388891.0,NaN,NaN,NaN,NaN,0.0,1.0,0.100000,0.900000
2,2404208.0,NaN,NaN,NaN,NaN,0.5,0.5,0.600000,0.400000
3,2388931.0,NaN,NaN,0.0,1.0,1.0,0.0,0.500000,0.500000
4,2392202.0,NaN,NaN,0.0,1.0,NaN,NaN,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...
22998,2400463.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000
22999,2398012.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000
23000,2397484.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000
23001,2397539.0,NaN,NaN,NaN,NaN,NaN,NaN,0.090909,0.909091


In [32]:
all_transition_probabilities['cluster'] = train_labels
all_transition_probabilities.to_csv('outputs/feb2021-data-transitions.csv')

In [34]:
all_transition_probabilities.groupby('cluster').std().drop(columns=['user_id']).\
                                                           to_csv('outputs/feb2021_clustering_stddev.csv')  
                        

In [49]:
all_transition_probabilities.drop('cluster', 1).isna().\
                    groupby(all_transition_probabilities.cluster).sum().reset_index().\
                    to_csv('outputs/feb2021_missing_counts.csv')


In [50]:
cluster_transition_probabilities

,cluster,count,"P(L, I, L)","P(L, I, H)","P(H, I, L)","P(H, I, H)","P(L, N, L)","P(L, N, H)","P(H, N, L)","P(H, N, H)"
0,0.0,324.0,0.666667,0.333333,0.428571,0.571429,0.655698,0.344302,0.217063,0.782937
1,1.0,195.0,0.716418,0.283582,NaN,NaN,1.000000,0.000000,1.000000,0.000000
2,2.0,1793.0,0.538462,0.461538,0.133903,0.866097,0.000000,1.000000,0.103642,0.896358
3,3.0,3649.0,0.977704,0.022296,NaN,NaN,1.000000,0.000000,0.254545,0.745455
4,4.0,5988.0,NaN,NaN,0.021218,0.978782,0.503876,0.496124,0.000000,1.000000
5,5.0,185.0,0.677419,0.322581,0.461538,0.538462,0.629865,0.370135,0.628651,0.371349
6,6.0,238.0,0.555556,0.444444,0.459459,0.540541,0.236581,0.763419,0.449583,0.550417
7,7.0,748.0,0.924528,0.075472,0.671642,0.328358,0.774875,0.225125,1.000000,0.000000
8,8.0,406.0,1.000000,0.000000,0.492754,0.507246,0.869411,0.130589,0.000000,1.000000
9,9.0,325.0,0.727273,0.272727,0.267606,0.732394,0.334690,0.665310,0.257303,0.742697


In [51]:
stddev_df = pd.read_csv('outputs/feb2021_clustering_stddev.csv')
p_cols = [col for col in stddev_df.columns if col.startswith('P')]
stddev_df = stddev_df.rename(columns={i: f'std-{i}' for i in p_cols})
stddev_df

,cluster,"std-P(L, I, L)","std-P(L, I, H)","std-P(H, I, L)","std-P(H, I, H)","std-P(L, N, L)","std-P(L, N, H)","std-P(H, N, L)","std-P(H, N, H)"
0,0,0.478091,0.478091,0.494398,0.494398,0.041150,0.041150,0.038247,0.038247
1,1,0.425258,0.425258,NaN,NaN,0.000000,0.000000,0.000000,0.000000
2,2,0.518875,0.518875,0.339708,0.339708,0.000000,0.000000,0.007269,0.007269
3,3,0.152383,0.152383,0.000000,0.000000,0.000000,0.000000,0.053237,0.053237
4,4,NaN,NaN,0.130560,0.130560,0.012699,0.012699,0.000000,0.000000
5,5,0.483725,0.483725,0.508391,0.508391,0.032559,0.032559,0.033208,0.033208
6,6,0.511310,0.511310,0.470929,0.470929,0.020985,0.020985,0.035495,0.035495
7,7,0.231837,0.231837,0.475017,0.475017,0.020815,0.020815,0.000000,0.000000
8,8,0.000000,0.000000,0.487550,0.487550,0.036794,0.036794,0.000000,0.000000
9,9,0.455842,0.455842,0.429703,0.429703,0.007362,0.007362,0.017231,0.017231


In [52]:
missing_df = pd.read_csv('outputs/feb2021_missing_counts.csv')
p_cols = [col for col in missing_df.columns if col.startswith('P')]
missing_df = missing_df.rename(columns={i: f'missing-{i}' for i in p_cols})
missing_df

,Unnamed: 0,cluster,user_id,"missing-P(L, I, L)","missing-P(L, I, H)","missing-P(H, I, L)","missing-P(H, I, H)","missing-P(L, N, L)","missing-P(L, N, H)","missing-P(H, N, L)","missing-P(H, N, H)"
0,0,0,0.0,288.0,288.0,290.0,290.0,3.0,3.0,0.0,0.0
1,1,1,0.0,96.0,96.0,195.0,195.0,0.0,0.0,0.0,0.0
2,2,2,0.0,1780.0,1780.0,1466.0,1466.0,0.0,0.0,0.0,0.0
3,3,3,0.0,2679.0,2679.0,3647.0,3647.0,0.0,0.0,3549.0,3549.0
4,4,4,0.0,5988.0,5988.0,5047.0,5047.0,5865.0,5865.0,0.0,0.0
5,5,5,0.0,156.0,156.0,159.0,159.0,0.0,0.0,0.0,0.0
6,6,6,0.0,220.0,220.0,206.0,206.0,0.0,0.0,0.0,0.0
7,7,7,0.0,609.0,609.0,682.0,682.0,0.0,0.0,0.0,0.0
8,8,8,0.0,377.0,377.0,344.0,344.0,0.0,0.0,0.0,0.0
9,9,9,0.0,303.0,303.0,258.0,258.0,0.0,0.0,0.0,0.0


In [53]:
big_summary_df = pd.merge(pd.merge(cluster_transition_probabilities, stddev_df ),
         missing_df.drop(columns=['Unnamed: 0', 'user_id']))

In [54]:
big_summary_df.to_csv('outputs/feb2021_clustering_summary.csv', index=False)

In [55]:
big_summary_df

,cluster,count,"P(L, I, L)","P(L, I, H)","P(H, I, L)","P(H, I, H)","P(L, N, L)","P(L, N, H)","P(H, N, L)","P(H, N, H)",...,"std-P(H, N, L)","std-P(H, N, H)","missing-P(L, I, L)","missing-P(L, I, H)","missing-P(H, I, L)","missing-P(H, I, H)","missing-P(L, N, L)","missing-P(L, N, H)","missing-P(H, N, L)","missing-P(H, N, H)"
0,0.0,324.0,0.666667,0.333333,0.428571,0.571429,0.655698,0.344302,0.217063,0.782937,...,0.038247,0.038247,288.0,288.0,290.0,290.0,3.0,3.0,0.0,0.0
1,1.0,195.0,0.716418,0.283582,NaN,NaN,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,96.0,96.0,195.0,195.0,0.0,0.0,0.0,0.0
2,2.0,1793.0,0.538462,0.461538,0.133903,0.866097,0.000000,1.000000,0.103642,0.896358,...,0.007269,0.007269,1780.0,1780.0,1466.0,1466.0,0.0,0.0,0.0,0.0
3,3.0,3649.0,0.977704,0.022296,NaN,NaN,1.000000,0.000000,0.254545,0.745455,...,0.053237,0.053237,2679.0,2679.0,3647.0,3647.0,0.0,0.0,3549.0,3549.0
4,4.0,5988.0,NaN,NaN,0.021218,0.978782,0.503876,0.496124,0.000000,1.000000,...,0.000000,0.000000,5988.0,5988.0,5047.0,5047.0,5865.0,5865.0,0.0,0.0
5,5.0,185.0,0.677419,0.322581,0.461538,0.538462,0.629865,0.370135,0.628651,0.371349,...,0.033208,0.033208,156.0,156.0,159.0,159.0,0.0,0.0,0.0,0.0
6,6.0,238.0,0.555556,0.444444,0.459459,0.540541,0.236581,0.763419,0.449583,0.550417,...,0.035495,0.035495,220.0,220.0,206.0,206.0,0.0,0.0,0.0,0.0
7,7.0,748.0,0.924528,0.075472,0.671642,0.328358,0.774875,0.225125,1.000000,0.000000,...,0.000000,0.000000,609.0,609.0,682.0,682.0,0.0,0.0,0.0,0.0
8,8.0,406.0,1.000000,0.000000,0.492754,0.507246,0.869411,0.130589,0.000000,1.000000,...,0.000000,0.000000,377.0,377.0,344.0,344.0,0.0,0.0,0.0,0.0
9,9.0,325.0,0.727273,0.272727,0.267606,0.732394,0.334690,0.665310,0.257303,0.742697,...,0.017231,0.017231,303.0,303.0,258.0,258.0,0.0,0.0,0.0,0.0


In [57]:
import pandas as pd
df = pd.read_csv('outputs/feb2021_clustering_summary.csv')
df

,cluster,count,"P(L, I, L)","P(L, I, H)","P(H, I, L)","P(H, I, H)","P(L, N, L)","P(L, N, H)","P(H, N, L)","P(H, N, H)",...,"std-P(H, N, L)","std-P(H, N, H)","missing-P(L, I, L)","missing-P(L, I, H)","missing-P(H, I, L)","missing-P(H, I, H)","missing-P(L, N, L)","missing-P(L, N, H)","missing-P(H, N, L)","missing-P(H, N, H)"
0,0.0,324.0,0.666667,0.333333,0.428571,0.571429,0.655698,0.344302,0.217063,0.782937,...,0.038247,0.038247,288.0,288.0,290.0,290.0,3.0,3.0,0.0,0.0
1,1.0,195.0,0.716418,0.283582,NaN,NaN,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,96.0,96.0,195.0,195.0,0.0,0.0,0.0,0.0
2,2.0,1793.0,0.538462,0.461538,0.133903,0.866097,0.000000,1.000000,0.103642,0.896358,...,0.007269,0.007269,1780.0,1780.0,1466.0,1466.0,0.0,0.0,0.0,0.0
3,3.0,3649.0,0.977704,0.022296,NaN,NaN,1.000000,0.000000,0.254545,0.745455,...,0.053237,0.053237,2679.0,2679.0,3647.0,3647.0,0.0,0.0,3549.0,3549.0
4,4.0,5988.0,NaN,NaN,0.021218,0.978782,0.503876,0.496124,0.000000,1.000000,...,0.000000,0.000000,5988.0,5988.0,5047.0,5047.0,5865.0,5865.0,0.0,0.0
5,5.0,185.0,0.677419,0.322581,0.461538,0.538462,0.629865,0.370135,0.628651,0.371349,...,0.033208,0.033208,156.0,156.0,159.0,159.0,0.0,0.0,0.0,0.0
6,6.0,238.0,0.555556,0.444444,0.459459,0.540541,0.236581,0.763419,0.449583,0.550417,...,0.035495,0.035495,220.0,220.0,206.0,206.0,0.0,0.0,0.0,0.0
7,7.0,748.0,0.924528,0.075472,0.671642,0.328358,0.774875,0.225125,1.000000,0.000000,...,0.000000,0.000000,609.0,609.0,682.0,682.0,0.0,0.0,0.0,0.0
8,8.0,406.0,1.000000,0.000000,0.492754,0.507246,0.869411,0.130589,0.000000,1.000000,...,0.000000,0.000000,377.0,377.0,344.0,344.0,0.0,0.0,0.0,0.0
9,9.0,325.0,0.727273,0.272727,0.267606,0.732394,0.334690,0.665310,0.257303,0.742697,...,0.017231,0.017231,303.0,303.0,258.0,258.0,0.0,0.0,0.0,0.0


In [58]:
df = pd.read_csv('outputs/may_2020_clustering_summary.csv')
df

,cluster,count,"P(L, I, L)","P(L, I, H)","P(H, I, L)","P(H, I, H)","P(L, N, L)","P(L, N, H)","P(H, N, L)","P(H, N, H)",...,"std-P(H, N, L)","std-P(H, N, H)","missing-P(L, I, L)","missing-P(L, I, H)","missing-P(H, I, L)","missing-P(H, I, H)","missing-P(L, N, L)","missing-P(L, N, H)","missing-P(H, N, L)","missing-P(H, N, H)"
0,0.0,46.0,1.000000,0.000000,NaN,NaN,0.791016,0.208984,0.792593,0.207407,...,0.045287,0.045287,28.0,28.0,44.0,44.0,0.0,0.0,0.0,0.0
1,1.0,135.0,0.142857,0.857143,0.272727,0.727273,0.333333,0.666667,0.080764,0.919236,...,0.028978,0.028978,128.0,128.0,124.0,124.0,0.0,0.0,0.0,0.0
2,2.0,96.0,0.250000,0.750000,0.470588,0.529412,0.579198,0.420802,0.238489,0.761511,...,0.029482,0.029482,92.0,92.0,79.0,79.0,0.0,0.0,0.0,0.0
3,3.0,248.0,0.400000,0.600000,0.083333,0.916667,0.000000,1.000000,0.075598,0.924402,...,0.002689,0.002689,243.0,243.0,212.0,212.0,0.0,0.0,0.0,0.0
4,4.0,60.0,0.750000,0.250000,0.727273,0.272727,0.396471,0.603529,0.570930,0.429070,...,0.047541,0.047541,56.0,56.0,49.0,49.0,0.0,0.0,0.0,0.0
5,5.0,122.0,0.625000,0.375000,0.454545,0.545455,0.262931,0.737069,0.253499,0.746501,...,0.029371,0.029371,114.0,114.0,111.0,111.0,0.0,0.0,0.0,0.0
6,6.0,47.0,0.941176,0.058824,NaN,NaN,0.914259,0.085741,0.329365,0.670635,...,0.043659,0.043659,30.0,30.0,45.0,45.0,0.0,0.0,6.0,6.0
7,7.0,65.0,NaN,NaN,0.470588,0.529412,0.724790,0.275210,0.071273,0.928727,...,0.042394,0.042394,65.0,65.0,48.0,48.0,0.0,0.0,0.0,0.0
8,8.0,118.0,0.600000,0.400000,0.826087,0.173913,0.606061,0.393939,0.447263,0.552737,...,0.037724,0.037724,103.0,103.0,95.0,95.0,0.0,0.0,0.0,0.0
9,9.0,26.0,0.333333,0.666667,0.750000,0.250000,0.072072,0.927928,0.389961,0.610039,...,0.034113,0.034113,23.0,23.0,22.0,22.0,0.0,0.0,0.0,0.0
